<a href="https://colab.research.google.com/github/ElmaNababan08/repository_skripsi/blob/main/data_karangasem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ANALISIS SENTIMEN TERHADAP DAYA TARIK WISATA KABUPATEN KARANGASEM MENGGUNAKAN MODEL INDOBERT

In [2]:
#install library google-play-scraper
!pip install emoji
!pip install PySastrawi
!pip -q install transformers datasets evaluate scikit-learn accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 608.4/608.4 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.6/210.6 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.6 MB/s eta 0:00:00


In [3]:
# import semua lib
import pandas as pd
import numpy as np
import  re, emoji
import nltk
from nltk.corpus import stopwords
import json
import ast
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import transformers
import torch
import evaluate
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report
from collections import Counter
from transformers import (
    AutoTokenizer, AutoModelForSequenceClassification,
    Trainer, TrainingArguments
)
from datasets import Dataset
import inspect
import os
import ast
from IPython.display import display
import inspect
from collections import Counter
from torch import nn
import evaluate

os.environ["TOKENIZERS_PARALLELISM"] = "false"  # hindari deadlock tokenizers
os.environ["PYTHONWARNINGS"] = "ignore"        # opsional, biar bersih

# Kurangi kontensi CPU
torch.set_num_threads(1)

In [ ]:
df = pd.read_csv('/content/data labeling keseluruhan wisata - Sheet1.csv', encoding='latin1')
df.head()

,Taman Ujung,Label,textTranslated
0,NaN,P,"Indah, tetapi panas karena terbuka"
1,NaN,P,"Tempat yang indah dan damai, sempurna untuk be..."
2,NaN,P,Tempat yang indah dan tenang untuk dikunjungi....
3,NaN,P,"Istana baru, sangat cantik dan belum terlalu r..."
4,NaN,P,"Bekas istana musim panas seorang raja, tempat ..."


# Step 2: Pre-Processing data







1. Cleaning

In [ ]:
# --- SETUP

# Baca data
df = pd.read_csv('/content/data labeling keseluruhan wisata - Sheet1.csv', encoding='latin1')

# 1) Normalkan nama kolom (hapus spasi depan/belakang, ganti spasi jadi underscore)
df.columns = (df.columns
              .str.strip()
              .str.replace(r'\s+', '_', regex=True))

# 2) Tentukan kolom teks secara robust (mis. "textTranslated", "Text Translated", dsb.)
candidates = [c for c in df.columns if c.lower().replace('_','') == 'texttranslated']
if not candidates:
    raise ValueError(f"Tidak menemukan kolom 'textTranslated'. Kolom tersedia: {list(df.columns)}")
TEXT_COL = candidates[0]
print("Kolom terdeteksi untuk teks:", TEXT_COL)

# 3) Hapus missing/duplikat HANYA berdasar kolom teks (kolom lain dibiarkan)
before = len(df)
df = df[~df[TEXT_COL].isna()].drop_duplicates(subset=[TEXT_COL]).copy()
print(f"Rows: {before} -> {len(df)} setelah drop NA/duplikat di kolom teks.")

# 4) Fungsi untuk membersihkan teks (tanpa menghapus baris)
def bersihkan_teks(text: str) -> str:
    text = str(text)

    # Hapus link
    text = re.sub(r'http\S+|www\.\S+', '', text)

    # Hapus kata iklan
    pattern_iklan = r'RESULT|SHIO|JACKPOT|Para Pemenang|SYAIR|ANGKA'
    text = re.sub(pattern_iklan, '', text, flags=re.IGNORECASE)

    # Rapatkan simbol berulang (!!!, ???, *** -> !, ?, *)
    text = re.sub(r'([?!\*\"\'\.\,]){3,}', r'\1', text)

    # Cleaning tambahan
    text = re.sub(r'@\w+', '', text)                 # mention
    text = re.sub(r'#\w+', '', text)                 # hashtag
    text = emoji.replace_emoji(text, replace='')     # emoji
    text = re.sub(r'\d+', '', text)                  # angka
    text = re.sub(r'[^\w\s]', '', text)              # tanda baca non-alfanum
    text = re.sub(r'\s+', ' ', text)                 # spasi ganda -> satu
    return text.strip()

# 5) Terapkan ke kolom TEXT_COL TANPA menghapus baris
df['cleaning_data'] = df[TEXT_COL].apply(bersihkan_teks)

# 6) Pratinjau
print("=== PRATINJAU ===")
print(df[[TEXT_COL, 'cleaning_data']].head(10))

# 7) Simpan
df.to_csv('final_cleaning_data.csv', index=False, encoding='utf-8')
print("\nData cleaning selesai → 'final_cleaning_data.csv'")
display(df[['textTranslated', 'cleaning_data']].head(10))



Kolom terdeteksi untuk teks: textTranslated
Rows: 22895 -> 21438 setelah drop NA/duplikat di kolom teks.
=== PRATINJAU ===
                                      textTranslated  \
0                 Indah, tetapi panas karena terbuka   
1  Tempat yang indah dan damai, sempurna untuk be...   
2  Tempat yang indah dan tenang untuk dikunjungi....   
3  Istana baru, sangat cantik dan belum terlalu r...   
4  Bekas istana musim panas seorang raja, tempat ...   
5  Beginilah penampakan istana. Pemandangan dari ...   
6  Biaya masuk 100k untuk melihat ikan mati... Ba...   
7  Taman Ujung adalah permata tersembunyi yang sa...   
8  Anda mungkin merasa itu hanyalah tempat lain s...   
9          Suatu keajaiban, dan tidak terlalu padat.   

                                       cleaning_data  
0                  Indah tetapi panas karena terbuka  
1  Tempat yang indah dan damai sempurna untuk ber...  
2  Tempat yang indah dan tenang untuk dikunjungi ...  
3  Istana baru sangat cantik dan belum t

,textTranslated,cleaning_data
0,"Indah, tetapi panas karena terbuka",Indah tetapi panas karena terbuka
1,"Tempat yang indah dan damai, sempurna untuk be...",Tempat yang indah dan damai sempurna untuk ber...
2,Tempat yang indah dan tenang untuk dikunjungi....,Tempat yang indah dan tenang untuk dikunjungi ...
3,"Istana baru, sangat cantik dan belum terlalu r...",Istana baru sangat cantik dan belum terlalu ramai
4,"Bekas istana musim panas seorang raja, tempat ...",Bekas istana musim panas seorang raja tempat y...
5,Beginilah penampakan istana. Pemandangan dari ...,Beginilah penampakan istana Pemandangan dari a...
6,Biaya masuk 100k untuk melihat ikan mati... Ba...,Biaya masuk k untuk melihat ikan mati Bali itu...
7,Taman Ujung adalah permata tersembunyi yang sa...,Taman Ujung adalah permata tersembunyi yang sa...
8,Anda mungkin merasa itu hanyalah tempat lain s...,Anda mungkin merasa itu hanyalah tempat lain s...
9,"Suatu keajaiban, dan tidak terlalu padat.",Suatu keajaiban dan tidak terlalu padat


2. Case Folding

In [ ]:
# 1. Import library pandas

# ============================
# PROSES CASE FOLDING
# ============================

# 2. Membaca dataset final_cleaning_data.csv
df = pd.read_csv('/content/final_cleaning_data.csv')

# 3. Fungsi case folding: ubah huruf menjadi huruf kecil
def case_folding(text):
    return str(text).lower()

# 4. Terapkan fungsi case folding ke kolom cleaning_data
df['case_folding'] = df['cleaning_data'].apply(case_folding)

# 5. Simpan hasilnya ke file baru (opsional)
df.to_csv('data_casefolding.csv', index=False)

# 6. Tampilkan hasil sebelum dan sesudah case folding (10 contoh)
from IPython.display import display

print("\n=== TABEL HASIL CASEFOLDING (SEBELUM & SESUDAH) ===")
display(df[['cleaning_data', 'case_folding']].head(10))



=== TABEL HASIL CASEFOLDING (SEBELUM & SESUDAH) ===


,cleaning_data,case_folding
0,Indah tetapi panas karena terbuka,indah tetapi panas karena terbuka
1,Tempat yang indah dan damai sempurna untuk ber...,tempat yang indah dan damai sempurna untuk ber...
2,Tempat yang indah dan tenang untuk dikunjungi ...,tempat yang indah dan tenang untuk dikunjungi ...
3,Istana baru sangat cantik dan belum terlalu ramai,istana baru sangat cantik dan belum terlalu ramai
4,Bekas istana musim panas seorang raja tempat y...,bekas istana musim panas seorang raja tempat y...
5,Beginilah penampakan istana Pemandangan dari a...,beginilah penampakan istana pemandangan dari a...
6,Biaya masuk k untuk melihat ikan mati Bali itu...,biaya masuk k untuk melihat ikan mati bali itu...
7,Taman Ujung adalah permata tersembunyi yang sa...,taman ujung adalah permata tersembunyi yang sa...
8,Anda mungkin merasa itu hanyalah tempat lain s...,anda mungkin merasa itu hanyalah tempat lain s...
9,Suatu keajaiban dan tidak terlalu padat,suatu keajaiban dan tidak terlalu padat


3. Stopword Removal

In [ ]:
# 1. Import library yang dibutuhkan

# 2. Download stopwords bahasa Indonesia dari NLTK
nltk.download('stopwords')

# 3. Baca dataset hasil case folding
df = pd.read_csv('/content/data_casefolding.csv')

stop_words = set(stopwords.words('indonesian'))
# .union(additional_stopwords)

# 5. Fungsi untuk hapus stopword tanpa tokenisasi eksplisit
def remove_stopwords(text):
    text = str(text).lower()
    text = re.sub(r'[^\w\s]', ' ', text)
    words = text.split()
    filtered = [word for word in words if word not in stop_words]
    return " ".join(filtered)

# 6. Terapkan fungsi ke kolom 'case_folding'
df['stopword_removal'] = df['case_folding'].apply(remove_stopwords)

# 7. Simpan hasil ke file baru
df.to_csv('data_stopword_removal.csv', index=False)

# 8. Tampilkan hasil sebelum dan sesudah stopword removal
print("✅ Stopword removal selesai!")
display(df[['case_folding', 'stopword_removal']].head(10))


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


✅ Stopword removal selesai!


,case_folding,stopword_removal
0,indah tetapi panas karena terbuka,indah panas terbuka
1,tempat yang indah dan damai sempurna untuk ber...,indah damai sempurna berjalanjalan menikmati a...
2,tempat yang indah dan tenang untuk dikunjungi ...,indah tenang dikunjungi ramai
3,istana baru sangat cantik dan belum terlalu ramai,istana cantik ramai
4,bekas istana musim panas seorang raja tempat y...,bekas istana musim panas raja keramaian turis ...
5,beginilah penampakan istana pemandangan dari a...,penampakan istana pemandangan sungguh menakjub...
6,biaya masuk k untuk melihat ikan mati bali itu...,biaya masuk k ikan mati bali tipuan nggak
7,taman ujung adalah permata tersembunyi yang sa...,taman ujung permata tersembunyi dikenal indah ...
8,anda mungkin merasa itu hanyalah tempat lain s...,puncak keajaiban pemandangan taman gunung laut...
9,suatu keajaiban dan tidak terlalu padat,keajaiban padat


4. Tokenize

In [ ]:
# 1. Import library yang dibutuhkan

# ============================
# PROSES TOKENIZING
# ============================

# 2. Baca dataset hasil stopword removal
df = pd.read_csv('/content/data_stopword_removal.csv')

# 3. Isi kolom stopword_removal dengan string kosong jika ada NaN
df['stopword_removal'] = df['stopword_removal'].fillna('')

# 4. Fungsi tokenize (contoh sederhana)
def tokenize(text):
    if not isinstance(text, str):
        return []
    return text.split()

# 5. Terapkan fungsi tokenize ke kolom 'stopword_removal'
df['tokens'] = df['stopword_removal'].apply(tokenize)

# 6. Simpan hasil ke file baru
df.to_csv('hasil_tokenizing.csv', index=False)

# 7. Tampilkan hasil sebelum dan sesudah tokenize
print("✅ Tokenizing selesai!")
from IPython.display import display
display(df[['stopword_removal', 'tokens']].head(10))


✅ Tokenizing selesai!


,stopword_removal,tokens
0,indah panas terbuka,"[indah, panas, terbuka]"
1,indah damai sempurna berjalanjalan menikmati a...,"[indah, damai, sempurna, berjalanjalan, menikm..."
2,indah tenang dikunjungi ramai,"[indah, tenang, dikunjungi, ramai]"
3,istana cantik ramai,"[istana, cantik, ramai]"
4,bekas istana musim panas raja keramaian turis ...,"[bekas, istana, musim, panas, raja, keramaian,..."
5,penampakan istana pemandangan sungguh menakjub...,"[penampakan, istana, pemandangan, sungguh, men..."
6,biaya masuk k ikan mati bali tipuan nggak,"[biaya, masuk, k, ikan, mati, bali, tipuan, ng..."
7,taman ujung permata tersembunyi dikenal indah ...,"[taman, ujung, permata, tersembunyi, dikenal, ..."
8,puncak keajaiban pemandangan taman gunung laut...,"[puncak, keajaiban, pemandangan, taman, gunung..."
9,keajaiban padat,"[keajaiban, padat]"


5. Normalize

In [ ]:
# 1. Import library yang dibutuhkan

# ============================
# PROSES NORMALISASI SLANG
# ============================

# 2. Upload slang-indo.json dari lokal (untuk Colab)
from google.colab import files
uploaded = files.upload()

# 3. Baca dataset hasil tokenizing
df = pd.read_csv('/content/hasil_tokenizing.csv')
df['tokens'] = df['tokens'].apply(ast.literal_eval)

# 4. Baca kamus slang dari file JSON
with open('/content/slangwords.txt', 'r', encoding='utf-8') as f:
    slang_dict = json.load(f)   # pastikan slangwords.txt isinya JSON valid

# 5. Fungsi normalisasi slang
def normalize_tokens(tokens):
    return [slang_dict.get(token.lower(), token) for token in tokens]

# 6. Terapkan normalisasi ke kolom 'tokens'
df['normalized'] = df['tokens'].apply(normalize_tokens)

# 7. Simpan hasil ke file baru
df.to_csv('hasil_normalization.csv', index=False)

# 8. Tampilkan hasil sebelum dan sesudah normalisasi
print("✅ Normalized selesai!")
from IPython.display import display
display(df[['tokens', 'normalized']].head(10))


Saving slangwords.txt to slangwords.txt
✅ Normalized selesai!


,tokens,normalized
0,"[indah, panas, terbuka]","[indah, panas, terbuka]"
1,"[indah, damai, sempurna, berjalanjalan, menikm...","[indah, damai, sempurna, berjalanjalan, menikm..."
2,"[indah, tenang, dikunjungi, ramai]","[indah, tenang, dikunjungi, ramai]"
3,"[istana, cantik, ramai]","[istana, cantik, ramai]"
4,"[bekas, istana, musim, panas, raja, keramaian,...","[bekas, istana, musim, panas, raja, keramaian,..."
5,"[penampakan, istana, pemandangan, sungguh, men...","[penampakan, istana, pemandangan, sungguh, men..."
6,"[biaya, masuk, k, ikan, mati, bali, tipuan, ng...","[biaya, masuk, ke, ikan, mati, bali, tipuan, t..."
7,"[taman, ujung, permata, tersembunyi, dikenal, ...","[taman, ujung, permata, tersembunyi, dikenal, ..."
8,"[puncak, keajaiban, pemandangan, taman, gunung...","[puncak, keajaiban, pemandangan, taman, gunung..."
9,"[keajaiban, padat]","[keajaiban, padat]"


6. Stemming

In [ ]:
# 1. Install dan import library yang dibutuhkan

# 2. Buat stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# 3. Baca file hasil normalisasi
df = pd.read_csv('/content/hasil_normalization.csv')
df['normalized'] = df['normalized'].apply(ast.literal_eval)

# 4. Fungsi stemming untuk setiap token
def stem_tokens(tokens):
    return [stemmer.stem(token) for token in tokens]

# 5. Terapkan stemming ke kolom 'normalized'
df['stemmed'] = df['normalized'].apply(stem_tokens)

# 6. Simpan hasil ke file baru
df.to_csv('hasil_stemming.csv', index=False)

# 7. Tampilkan hasil sebelum dan sesudah stemming
print("✅ Stemming selesai!")
display(df[['normalized', 'stemmed']].head(10))


✅ Stemming selesai!


,normalized,stemmed
0,"[indah, panas, terbuka]","[indah, panas, buka]"
1,"[indah, damai, sempurna, berjalanjalan, menikm...","[indah, damai, sempurna, berjalanjalan, nikmat..."
2,"[indah, tenang, dikunjungi, ramai]","[indah, tenang, kunjung, ramai]"
3,"[istana, cantik, ramai]","[istana, cantik, ramai]"
4,"[bekas, istana, musim, panas, raja, keramaian,...","[bekas, istana, musim, panas, raja, ramai, tur..."
5,"[penampakan, istana, pemandangan, sungguh, men...","[tampak, istana, pandang, sungguh, takjub, sis..."
6,"[biaya, masuk, ke, ikan, mati, bali, tipuan, t...","[biaya, masuk, ke, ikan, mati, bal, tipu, tidak]"
7,"[taman, ujung, permata, tersembunyi, dikenal, ...","[taman, ujung, permata, sembunyi, kenal, indah..."
8,"[puncak, keajaiban, pemandangan, taman, gunung...","[puncak, ajaib, pandang, taman, gunung, laut, ..."
9,"[keajaiban, padat]","[ajaib, padat]"


# Step 3 : Fine Tuning IndoBERT

In [1]:
# =========================
# 0) Install & Import
# =========================
import os, inspect, ast, json
import numpy as np
import pandas as pd
from collections import Counter

import torch
from torch import nn

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report

from datasets import Dataset
import evaluate

from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer
)

# =========================
# 1) Load Data
# =========================
PATH = "/content/hasil_stemming.csv"  # ganti dengan path file kamu
TEXT_COL = "cleaning_data"
LABEL_COL = "Label"

df = pd.read_csv(PATH, encoding="utf-8")
df = df[[TEXT_COL, LABEL_COL]].dropna().rename(columns={TEXT_COL: "text", LABEL_COL: "label_raw"})
df = df[df["label_raw"].isin(["P", "N", "O"])].copy()

# Mapping label → ID
label2id = {"N": 0, "O": 1, "P": 2}
id2label = {v: k for k, v in label2id.items()}
df["label"] = df["label_raw"].map(label2id)

print("Distribusi label:", Counter(df["label_raw"]))

# =========================
# 2) Tokenizer
# =========================
MODEL_NAME = "indobenchmark/indobert-base-p1"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

def tokenize_fn(batch):
    return tokenizer(batch["text"], truncation=True, max_length=256, padding="max_length")

# =========================
# 3) Metrics
# =========================
acc_metric = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    return {
        "accuracy": acc_metric.compute(predictions=preds, references=labels)["accuracy"],
        "f1_macro": f1_metric.compute(predictions=preds, references=labels, average="macro")["f1"],
        "f1_weighted": f1_metric.compute(predictions=preds, references=labels, average="weighted")["f1"],
    }

# =========================
# 4) Helper: TrainingArguments (tanpa autosave)
# =========================
def make_training_args(out_dir: str) -> TrainingArguments:
    base = dict(
        output_dir=out_dir,
        learning_rate=3e-5,
        per_device_train_batch_size=8,
        per_device_eval_batch_size=8,
        num_train_epochs=4,
        weight_decay=0.01,
        warmup_ratio=0.06,
        lr_scheduler_type="cosine",
        gradient_accumulation_steps=2,
        fp16=torch.cuda.is_available(),
        seed=42,
        optim="adamw_torch",
        adam_beta1=0.9,
        adam_beta2=0.999,
        adam_epsilon=1e-8,
        max_grad_norm=1.0,
        report_to="none",

        # ⬇️ penting supaya gak hang
        dataloader_num_workers=0,        # <— single-process loading
        dataloader_pin_memory=False,     # <— matikan pinning (lebih aman di Kaggle)
        dataloader_persistent_workers=False,

        logging_dir=None,
        logging_steps=100,

        # tidak ada autosave
        save_strategy="no",
        eval_strategy="no",
        save_safetensors=True,
        save_only_model=True,
        load_best_model_at_end=False,
    )
    return TrainingArguments(**base)


# =========================
# 5) 10-Fold Cross Validation (save model sekali/akhir fold)
# =========================
k = 10
skf = StratifiedKFold(n_splits=k, shuffle=True, random_state=42)

all_reports = []
fold = 1

for train_idx, test_idx in skf.split(df["text"], df["label"]):
    print(f"\n===== Fold {fold}/{k} =====")

    train_df = df.iloc[train_idx]
    test_df = df.iloc[test_idx]

    # Dataset
    train_ds = Dataset.from_pandas(train_df[["text", "label"]], preserve_index=False)
    test_ds  = Dataset.from_pandas(test_df[["text", "label"]], preserve_index=False)

    train_ds = train_ds.map(tokenize_fn, batched=True)
    test_ds  = test_ds.map(tokenize_fn, batched=True)

    train_ds = train_ds.remove_columns(["text"])
    test_ds  = test_ds.remove_columns(["text"])

    # Class weights
    counts = train_df["label"].value_counts().sort_index().values
    class_weights = (1.0 / counts) * (counts.sum() / len(counts))
    class_weights = torch.tensor(class_weights, dtype=torch.float)

    # Model
    model = AutoModelForSequenceClassification.from_pretrained(
        MODEL_NAME, num_labels=3, id2label=id2label, label2id=label2id
    )

    # Custom Trainer
    class WeightedTrainer(Trainer):
        def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
            labels = inputs.get("labels")
            outputs = model(**inputs)
            logits = outputs.get("logits")
            loss_fct = nn.CrossEntropyLoss(weight=class_weights.to(logits.device))
            loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))
            return (loss, outputs) if return_outputs else loss

    out_dir = f"/kaggle/working/indoBERT-kfold/fold_{fold}"
    args = make_training_args(out_dir)

    trainer = WeightedTrainer(
        model=model,
        args=args,
        train_dataset=train_ds,
        eval_dataset=None,         # evaluasi manual setelah training
        tokenizer=tokenizer,
        compute_metrics=None,      # tak dibutuhkan saat train
    )

    # === Train ===
    trainer.train()

    # === Simpan model SEKALI di akhir fold ===
    os.makedirs(out_dir, exist_ok=True)
    trainer.save_model(out_dir)          # hanya weights+config+tokenizer (save_only_model jika didukung)
    if hasattr(trainer, "optimizer") and trainer.optimizer is not None:
        del trainer.optimizer
    if hasattr(trainer, "lr_scheduler") and trainer.lr_scheduler is not None:
        del trainer.lr_scheduler

    # === Evaluasi per fold (tanpa menyimpan apapun) ===
    # gunakan Trainer temporer dengan eval_dataset
    eval_trainer = Trainer(
        model=model,
        args=TrainingArguments(
            output_dir=out_dir,                   # tidak dipakai untuk save
            per_device_eval_batch_size=32,
            fp16=torch.cuda.is_available(),
            report_to="none",
            save_strategy="no",
            eval_strategy="no",  # diganti sesuai versi 4.53.3
            logging_dir=None,
            dataloader_num_workers=0,
            dataloader_pin_memory=False,
            dataloader_persistent_workers=False
        ),
        eval_dataset=test_ds,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics,
    )

    preds_output = eval_trainer.predict(test_ds)
    y_pred = np.argmax(preds_output.predictions, axis=1)
    y_true = np.array(test_df["label"])

    report = classification_report(
        y_true, y_pred,
        target_names=[id2label[i] for i in range(3)],
        output_dict=True
    )
    all_reports.append(report)

    print(f"\n=== Fold {fold} Selesai ===")
    print(f"Akurasi: {report['accuracy']:.4f}")
    print(f"F1-macro: {report['macro avg']['f1-score']:.4f}")

    # Bebaskan memori sebelum lanjut ke fold berikutnya
    del eval_trainer, trainer, model
    torch.cuda.empty_cache()

    fold += 1

# =========================
# 6) Hasil Rata-Rata
# =========================
mean_acc = np.mean([r["accuracy"] for r in all_reports])
mean_f1  = np.mean([r["macro avg"]["f1-score"] for r in all_reports])
print("\n===== HASIL RATA-RATA 10 FOLD =====")
print(f"Akurasi rata-rata: {mean_acc:.4f}")
print(f"F1-macro rata-rata: {mean_f1:.4f}")

ModuleNotFoundError: No module named 'evaluate'

# Step 3 : Fine Tuning IndoBERT

In [5]:
# =========================
# 0) Install & Import
# =========================
import os, inspect, ast, json
import numpy as np
import pandas as pd
from collections import Counter

import torch
from torch import nn

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report

from datasets import Dataset
import evaluate

from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer
)

# =========================
# 1) Load Data
# =========================
PATH = "/content/hasil_stemming_balanced.csv"  # ganti dengan path file kamu
TEXT_COL = "cleaning_data"
LABEL_COL = "Label"

df = pd.read_csv(PATH, encoding="utf-8")
df = df[[TEXT_COL, LABEL_COL]].dropna().rename(columns={TEXT_COL: "text", LABEL_COL: "label_raw"})
df = df[df["label_raw"].isin(["P", "N", "O"])].copy()

# Mapping label → ID
label2id = {"N": 0, "O": 1, "P": 2}
id2label = {v: k for k, v in label2id.items()}
df["label"] = df["label_raw"].map(label2id)

print("Distribusi label:", Counter(df["label_raw"]))

# =========================
# 2) Tokenizer
# =========================
MODEL_NAME = "indobenchmark/indobert-base-p1"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

def tokenize_fn(batch):
    return tokenizer(batch["text"], truncation=True, max_length=384)

# =========================
# 3) Metrics
# =========================
acc_metric = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    return {
        "accuracy": acc_metric.compute(predictions=preds, references=labels)["accuracy"],
        "f1_macro": f1_metric.compute(predictions=preds, references=labels, average="macro")["f1"],
        "f1_weighted": f1_metric.compute(predictions=preds, references=labels, average="weighted")["f1"],
    }

# =========================
# 4) Helper: TrainingArguments (tanpa autosave)
# =========================
def make_training_args(out_dir: str) -> TrainingArguments:
    base = dict(
        output_dir=out_dir,
        learning_rate=3e-5,
        per_device_train_batch_size=8,
        per_device_eval_batch_size=32,
        num_train_epochs=6,
        weight_decay=0.01,
        warmup_ratio=0.1,
        lr_scheduler_type="cosine",
        gradient_accumulation_steps=2,
        fp16=torch.cuda.is_available(),
        seed=42,
        optim="adamw_torch",
        adam_beta1=0.9,
        adam_beta2=0.999,
        adam_epsilon=1e-8,
        max_grad_norm=1.0,
        report_to="none",

        # ⬇️ penting supaya gak hang
        dataloader_num_workers=0,        # <— single-process loading
        dataloader_pin_memory=False,     # <— matikan pinning (lebih aman di Kaggle)
        dataloader_persistent_workers=False,

        logging_dir=None,
        logging_steps=100,

        # tidak ada autosave
        save_strategy="no",
        eval_strategy="no",
        save_safetensors=True,
        save_only_model=True,
        load_best_model_at_end=False,
    )
    return TrainingArguments(**base)


# =========================
# 5) 10-Fold Cross Validation (save model sekali/akhir fold)
# =========================
k = 10
skf = StratifiedKFold(n_splits=k, shuffle=True, random_state=42)

all_reports = []
fold = 1

for train_idx, test_idx in skf.split(df["text"], df["label"]):
    print(f"\n===== Fold {fold}/{k} =====")

    train_df = df.iloc[train_idx]
    test_df = df.iloc[test_idx]

    # Dataset
    train_ds = Dataset.from_pandas(train_df[["text", "label"]], preserve_index=False)
    test_ds  = Dataset.from_pandas(test_df[["text", "label"]], preserve_index=False)

    train_ds = train_ds.map(tokenize_fn, batched=True)
    test_ds  = test_ds.map(tokenize_fn, batched=True)

    train_ds = train_ds.remove_columns(["text"])
    test_ds  = test_ds.remove_columns(["text"])

    # Class weights
    counts = train_df["label"].value_counts().sort_index().values
    class_weights = (1.0 / counts) * (counts.sum() / len(counts))
    class_weights = torch.tensor(class_weights, dtype=torch.float)

    # Model
    model = AutoModelForSequenceClassification.from_pretrained(
        MODEL_NAME, num_labels=3, id2label=id2label, label2id=label2id
    )

    # Custom Trainer
    class WeightedTrainer(Trainer):
        def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
            labels = inputs.get("labels")
            outputs = model(**inputs)
            logits = outputs.get("logits")
            loss_fct = nn.CrossEntropyLoss(weight=class_weights.to(logits.device))
            loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))
            return (loss, outputs) if return_outputs else loss

    out_dir = f"/kaggle/working/indoBERT-kfold/fold_{fold}"
    args = make_training_args(out_dir)

    trainer = WeightedTrainer(
        model=model,
        args=args,
        train_dataset=train_ds,
        eval_dataset=None,         # evaluasi manual setelah training
        tokenizer=tokenizer,
        compute_metrics=None,      # tak dibutuhkan saat train
    )

    # === Train ===
    trainer.train()

    # === Simpan model SEKALI di akhir fold ===
    os.makedirs(out_dir, exist_ok=True)
    trainer.save_model(out_dir)          # hanya weights+config+tokenizer (save_only_model jika didukung)
    if hasattr(trainer, "optimizer") and trainer.optimizer is not None:
        del trainer.optimizer
    if hasattr(trainer, "lr_scheduler") and trainer.lr_scheduler is not None:
        del trainer.lr_scheduler

    # === Evaluasi per fold (tanpa menyimpan apapun) ===
    # gunakan Trainer temporer dengan eval_dataset
    eval_trainer = Trainer(
        model=model,
        args=TrainingArguments(
            output_dir=out_dir,                   # tidak dipakai untuk save
            per_device_eval_batch_size=32,
            fp16=torch.cuda.is_available(),
            report_to="none",
            save_strategy="no",
            eval_strategy="no",  # diganti sesuai versi 4.53.3
            logging_dir=None,
            dataloader_num_workers=0,
            dataloader_pin_memory=False,
            dataloader_persistent_workers=False
        ),
        eval_dataset=test_ds,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics,
    )

    preds_output = eval_trainer.predict(test_ds)
    y_pred = np.argmax(preds_output.predictions, axis=1)
    y_true = np.array(test_df["label"])

    report = classification_report(
        y_true, y_pred,
        target_names=[id2label[i] for i in range(3)],
        output_dict=True
    )
    all_reports.append(report)

    print(f"\n=== Fold {fold} Selesai ===")
    print(f"Akurasi: {report['accuracy']:.4f}")
    print(f"F1-macro: {report['macro avg']['f1-score']:.4f}")

    # Bebaskan memori sebelum lanjut ke fold berikutnya
    del eval_trainer, trainer, model
    torch.cuda.empty_cache()

    fold += 1

# =========================
# 6) Hasil Rata-Rata
# =========================
mean_acc = np.mean([r["accuracy"] for r in all_reports])
mean_f1  = np.mean([r["macro avg"]["f1-score"] for r in all_reports])
print("\n===== HASIL RATA-RATA 10 FOLD =====")
print(f"Akurasi rata-rata: {mean_acc:.4f}")
print(f"F1-macro rata-rata: {mean_f1:.4f}")

Distribusi label: Counter({'P': 9100, 'N': 7000, 'O': 6948})

===== Fold 1/10 =====


Map:   0%|          | 0/20743 [00:00<?, ? examples/s]

Map:   0%|          | 0/2305 [00:00<?, ? examples/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at indobenchmark/indobert-base-p1 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipython-input-1795152815.py:156: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `WeightedTrainer.__init__`. Use `processing_class` instead.
  trainer = WeightedTrainer(


Step,Training Loss
100,1.037600
200,0.775500
300,0.677500
400,0.626300
500,0.620000
600,0.588600
700,0.594000
800,0.581100
900,0.590700
1000,0.564700


...

Map:   0%|          | 0/2304 [00:00<?, ? examples/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at indobenchmark/indobert-base-p1 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipython-input-1795152815.py:156: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `WeightedTrainer.__init__`. Use `processing_class` instead.
  trainer = WeightedTrainer(


Step,Training Loss
100,1.039000
200,0.750700
300,0.665500
400,0.612000
500,0.604500
600,0.609700
700,0.595200
800,0.595800
900,0.578100
1000,0.563900


/tmp/ipython-input-1795152815.py:178: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  eval_trainer = Trainer(



=== Fold 10 Selesai ===
Akurasi: 0.8889
F1-macro: 0.8885

===== HASIL RATA-RATA 10 FOLD =====
Akurasi rata-rata: 0.9006
F1-macro rata-rata: 0.9001


In [ ]:
!pip install evaluate